In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/MyDrive/Veri-Kazima/colab

/content/drive/MyDrive/Veri-Kazima/colab


In [ ]:
!pwd

/content/drive/MyDrive/Veri-Kazima/colab


In [ ]:
import pandas as pd
import json

def csv_to_alpaca(csv_file_path, output_json_path):
    """
    CSV dosyasını Alpaca formatına dönüştürür ve JSON dosyasına kaydeder.

    Args:
        csv_file_path (str): CSV dosyasının yolu.
        output_json_path (str): Oluşturulacak JSON dosyasının yolu.
    """

    # CSV dosyasını pandas DataFrame'ine oku
    df = pd.read_csv(csv_file_path)

    # Alpaca formatına dönüştür
    alpaca_data = []
    for index, row in df.iterrows():
        alpaca_entry = {
            "instruction": row["Baslik"],  # Başlık bilgisini "instruction" alanına ekle
            "input": row["Ozet"],  # Özet bilgisini "input" alanına ekle
            "output": row["Icerik"]  # İçerik bilgisini "output" alanına ekle
        }
        alpaca_data.append(alpaca_entry)

    # JSON dosyasına kaydet
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(alpaca_data, f, ensure_ascii=False, indent=4)

# Örnek kullanım
csv_file_path = "/content/drive/MyDrive/Veri-Kazima/Model_Eğitim/Tum-Veri-Seti-Nisan2024-10.csv"  # CSV dosyanızın adı
output_json_path = "1alpaca_format.json"  # Oluşturulacak JSON dosyasının adı
csv_to_alpaca(csv_file_path, output_json_path)

Model Eğitimi

In [ ]:
!pip install transformers torch peft datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="1alpaca_format.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = "YOUR_HUGGING_FACE_TOKEN"  # Replace with your actual token

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_4bit=True)  # Quantization ile bellek tasarrufu

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/mistralai/Mistral-7B-v0.1.
401 Client Error. (Request ID: Root=1-67b6298a-0abdd52640d7254b0ef6902e;39e33de3-0146-41d7-b22a-65142dcd8bf6)

Cannot access gated repo for url https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json.
Access to model mistralai/Mistral-7B-v0.1 is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],  # Kritik katmanlar
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,  # T4 için uygun
    gradient_accumulation_steps=4,   # Bellek optimizasyonu
    learning_rate=2e-4,             # 3B-7B için ideal
    num_train_epochs=3,
    fp16=True,                      # GPU desteği
    logging_steps=100,
    save_steps=500
)

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    tokenizer=tokenizer
)
trainer.train()

///

In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = "senin token"  # Replace with your actual token

In [ ]:
# Gerekli Kütüphaneler
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from google.colab import drive
import torch

# ----------------------------------------------
# 1. Google Drive Bağlantısı (Checkpoint'ler için)
# ----------------------------------------------
drive.mount('/content/drive')
CHECKPOINT_DIR = "/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints"  # Kayıt klasörü



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content/drive/My Drive/Veri-Kazima/colab


In [ ]:
%cd /content/drive/MyDrive/Veri-Kazima/colab

/content/drive/MyDrive/Veri-Kazima/colab


In [ ]:
!pwd

/content/drive/MyDrive/Veri-Kazima/colab


In [ ]:
# ----------------------------------------------
# 2. Model ve Tokenizer Yükleme
# ----------------------------------------------
model_name = "meta-llama/Meta-Llama-3-8B"  # Kendi modelinizi seçin
tokenizer = AutoTokenizer.from_pretrained(model_name, token="senin token")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,  # Bellek optimizasyonu
    device_map="auto"
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model-00001-of-00004.safetensors:  95%|#########4| 4.71G/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:


# ----------------------------------------------
# 3. LoRA Konfigürasyonu (Opsiyonel)
# ----------------------------------------------
peft_config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

In [ ]:
# ----------------------------------------------
# 4. Veri Setini Yükleme (Alpaca Formatında)
# ----------------------------------------------
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Veri-Kazima/colab/1alpaca_format.json")["train"]


In [ ]:
# ----------------------------------------------
# 5. Eğitim Ayarları (Checkpoint ile)
# ----------------------------------------------
training_args = TrainingArguments(
    output_dir=CHECKPOINT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,  # Toplam epoch
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",          # Her epoch sonunda kaydet
    save_total_limit=3,             # En son 3 checkpoint sakla
    logging_steps=50,
    resume_from_checkpoint=True,    # Otomatik devam et
    report_to="none"                # TensorBoard/WandB kullanmıyoruz
)


In [ ]:
#yeni
# ----------------------------------------------
# 5. Eğitim Ayarları (Checkpoint ile)
# ----------------------------------------------
training_args = TrainingArguments(
    output_dir=CHECKPOINT_DIR,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="epoch",
    save_total_limit=3,
    logging_steps=50,
    resume_from_checkpoint=True,  # Auto-find latest checkpoint
    report_to="none",
    remove_unused_columns=False
)

In [ ]:
# ----------------------------------------------
# 6. Trainer'ı Başlat
# ----------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

<ipython-input-28-b5c5da898a85>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#yeni

# ----------------------------------------------
# 6. Trainer'ı Başlat (Düzeltilmiş Data Collator)
# ----------------------------------------------
def custom_collator(batch):
    inputs = []
    labels = []
    for example in batch:
        # Instruction + Input'i birleştir
        text = f"### Instruction: {example['instruction']}\n### Input: {example['input']}\n### Output: "
        inputs.append(text)
        labels.append(example['output'])

    # Tokenize
    model_inputs = tokenizer(
        inputs,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

    # Labels için ayrı tokenize
    labels = tokenizer(
        labels,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=custom_collator  # Düzeltilmiş collator
)


<ipython-input-39-f4794b6104c8>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:


# ----------------------------------------------
# 7. Eğitimi Başlat veya Devam Ettir
# ----------------------------------------------
try:
    print("Son checkpoint bulunuyor...")
    trainer.train(resume_from_checkpoint=True)  # Son kayıttan devam et
except:
    print("Checkpoint bulunamadı, yeni eğitim başlatılıyor...")
    trainer.train()


In [ ]:
#yeni
# ----------------------------------------------
# 7. Eğitimi Başlat veya Devam Ettir (Düzeltilmiş)
# ----------------------------------------------
try:
    print("Son checkpoint aranıyor...")
    trainer.train(resume_from_checkpoint=True)  # Otomatik son checkpoint'i bul
except Exception as e:
    print(f"Hata: {e}\nYeni eğitim başlatılıyor...")
    trainer.train()

Son checkpoint aranıyor...
Hata: No valid checkpoint found in output directory (/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints)
Yeni eğitim başlatılıyor...


ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:

# ----------------------------------------------
# 8. Final Modeli Kaydet
# ----------------------------------------------
model.save_pretrained(f"{CHECKPOINT_DIR}/final_model")
tokenizer.save_pretrained(f"{CHECKPOINT_DIR}/final_model")
print("Eğitim tamamlandı!")

//

In [ ]:
!pip install transformers torch peft datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip uninstall bitsandbytes -y

Found existing installation: bitsandbytes 0.45.2
Uninstalling bitsandbytes-0.45.2:
  Successfully uninstalled bitsandbytes-0.45.2


In [ ]:
!pip install -U bitsandbytes # Upgrades bitsandbytes to the latest version

  Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl.metadata (5.8 kB)
Using cached bitsandbytes-0.45.2-py3-none-manylinux_2_24_x86_64.whl (69.7 MB)


In [ ]:
#cuda yok ise
!pip install cloud-tpu-client==0.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.4/120.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3
    Uninstalling protobuf-5.29.3:
      Successfully uninstalled protobuf-5.29.3
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.24.1
    Uninstalling google-api-core-2.24.1:
      Successfully uninstalled google-api-core-2.24.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 2.19.0 requires google-api-core<3.0.0dev,>=2.15.0, but you have google-api-core 1.34.1 which is incompatible.
tensorflow-metadata 1.1

In [ ]:
# ----------------------------------------------
# 0. Gerekli Kütüphaneler
# ----------------------------------------------
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from google.colab import drive
import torch
import os

# ----------------------------------------------
# 1. Google Drive Bağlantısı
# ----------------------------------------------
#drive.mount('/content/drive')
CHECKPOINT_DIR = "/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints"

In [ ]:
%cd /content/drive/MyDrive/Veri-Kazima/colab

/content/drive/MyDrive/Veri-Kazima/colab


In [ ]:
import os
os.environ["HUGGING_FACE_HUB_TOKEN"] = "senin token"  # Replace with your actual token

In [ ]:

# ----------------------------------------------
# 2. Tokenizer ve Padding Token Ayarları
# ----------------------------------------------
model_name = "google/gemma-2b" #"meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, token="senin token")

# Padding token ekleme (Mistral için gerekli)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
#cuda yok ise
# ----------------------------------------------
# 2. Tokenizer ve Padding Token Ayarları
# ----------------------------------------------
model_name = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_name, token="senin token")

# Padding token ekleme (Mistral için gerekli)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
#cuda yok ise
# 3. Model Yükleme (TPU Optimize Edilmiş)
# ----------------------------------------------
import torch_xla.core.xla_model as xm

# TPU cihazını seç
device = xm.xla_device()

# Modeli TPU'ya yükle (Quantization OLMADAN)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float32,  # TPU'lar float32 ile daha iyi çalışır
    device_map={"": device}      # TPU'ya yükle
)

# Token embeddings boyutunu güncelle
model.resize_token_embeddings(len(tokenizer))

# Modeli TPU'ya taşı
model = model.to(device)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Bad StatusOr access: RESOURCE_EXHAUSTED: Error allocating device buffer: Attempting to allocate 1.95G. That was not possible. There are 217.78M free.; (0x0x0_HBM0)

In [ ]:
# ----------------------------------------------
# 3. Model Yükleme
# ----------------------------------------------
from transformers import BitsAndBytesConfig

# Quantization ayarlarını yapılandır
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit kantizasyon kullan
    bnb_4bit_quant_type="nf4",  # Kantizasyon tipi
    bnb_4bit_use_double_quant=True,  # Çift kantizasyon kullan
)


# Modeli yükle
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Quantization ayarlarını geçir
    device_map="auto",
    torch_dtype=torch.float16
)
model.resize_token_embeddings(len(tokenizer))  # Yeni token için boyut ayarla

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Embedding(256000, 2048, padding_idx=0)

In [ ]:
# ----------------------------------------------
# 4. LoRA Konfigürasyonu
# ----------------------------------------------
peft_config = LoraConfig(
    r=16,#64
    lora_alpha=16,#64
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.001, #0.05
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

In [ ]:
# ----------------------------------------------
# 5. Veri Hazırlama (Düzeltilmiş)
# ----------------------------------------------
def format_dataset(example):
    return {
        "instruction": example["instruction"],
        "input": example["input"],
        "output": example["output"]
    }

dataset = load_dataset("json", data_files="/content/drive/MyDrive/Veri-Kazima/colab/1alpaca_format.json")["train"]
dataset = dataset.map(format_dataset)

In [ ]:
# ----------------------------------------------
# 6. Özelleştirilmiş Data Collator (DÜZELTİLMİŞ)
# ----------------------------------------------
def custom_collator(batch):
    # Instruction + Input + Output formatını oluştur
    texts = [
        f"### Instruction: {x['instruction']}\n### Input: {x['input']}\n### Output: {x['output']}"
        for x in batch
    ]

    # Tokenize işlemi (LABELS dahil)
    inputs = tokenizer(
        texts,
        padding="longest",
        truncation=True,
        max_length=64,#256
        return_tensors="pt"
    )

    # Labels = input_ids (Causal LM için)
    inputs["labels"] = inputs["input_ids"].clone()
    return inputs


# ----------------------------------------------
# 7. Eğitim Ayarları
# ----------------------------------------------
training_args = TrainingArguments(
    output_dir=  CHECKPOINT_DIR #"./output",
    per_device_train_batch_size=16,  # 2 -> 1 (bellek tasarrufu)
    gradient_accumulation_steps=1,   # 4 -> 8
    num_train_epochs=1,              # Toplam 3 epoch
    learning_rate=1e-4,              # 2e-4 -> 1e-4 (stabilite için)
    fp16=True,
    save_strategy="epoch",
    logging_steps=200,
    remove_unused_columns=False,
    report_to="none",
    optim="adamw_8bit"               # 8-bit optimizasyon
)

In [ ]:
# ----------------------------------------------
# 8. Trainer (DÜZELTİLMİŞ)
# ----------------------------------------------
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=custom_collator
)

<ipython-input-47-edf124c7f611>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Step,Training Loss


In [ ]:
# ----------------------------------------------
# 9. Eğitimi Başlat
# ----------------------------------------------
try:
    print("Checkpoint aranıyor...")
    trainer.train(resume_from_checkpoint=True)
except Exception as e:
    print(f"Hata: {str(e)[:200]}...\nYeni eğitim başlatılıyor...")
    trainer.train()

# ----------------------------------------------
# 10. Modeli Kaydet
# ----------------------------------------------
model.save_pretrained(f"{CHECKPOINT_DIR}/final_model")
tokenizer.save_pretrained(f"{CHECKPOINT_DIR}/final_model")

Checkpoint aranıyor...
Hata: No valid checkpoint found in output directory (./output)...
Yeni eğitim başlatılıyor...


Step,Training Loss
200,2.406200
400,2.403000
600,2.371500
800,2.362900
1000,2.343700
1200,2.340100
1400,2.298900
1600,2.247500
1800,2.206900
2000,2.228600


Step,Training Loss
200,2.406200
400,2.403000
600,2.371500
800,2.362900
1000,2.343700
1200,2.340100
1400,2.298900
1600,2.247500
1800,2.206900
2000,2.228600


('/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints/final_model/tokenizer.model',
 '/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints/final_model/added_tokens.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/llama_checkpoints/final_model/tokenizer.json')

//

In [ ]:
!pwd

/content/drive/MyDrive/Veri-Kazima/colab


In [ ]:
# Eğitim sonunda kaydetme
model.save_pretrained("/content/drive/MyDrive/Veri-Kazima/colab/deniz")


In [ ]:
tokenizer.save_pretrained("/content/drive/MyDrive/Veri-Kazima/colab/deniz")

('/content/drive/MyDrive/Veri-Kazima/colab/deniz/tokenizer_config.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz/special_tokens_map.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz/tokenizer.model',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz/added_tokens.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz/tokenizer.json')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Model ve tokenizer'ı yükle
model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114"  # Kendi model yolunuz
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

# CUDA yoksa CPU'ya taşı
if not torch.cuda.is_available():
    model = model.to("cpu")

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_new_tokens=200,
    temperature=0.8,
    top_p=0.95,
    repetition_penalty=1.1
)

In [ ]:
# Futbol haberi üretme örneği
prompt = """Fenerbahçe will be without the services of their star striker, Cenk Tosun, for the next two matches.

The Turkish club announced on their official website that the striker will be out for two weeks due to a muscle injury.

The club also announced that"""

# Metin üret
result = generator(prompt)
print(result[0]['generated_text'])

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ----------------------------------------------
# 1. Model ve Tokenizer Yükleme (Yerel Yol)
# ----------------------------------------------
model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114"  # Klasör yolu olmalı

# Tokenizer'ı yükle (mutlaka tokenizer klasörü olmalı)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Modeli yükle (config.json ve model dosyaları kontrol edilmeli)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",  # Otomatik CUDA/CPU atama
    torch_dtype=torch.float16  # Bellek optimizasyonu
)

# ----------------------------------------------
# 2. Modeli Kullanma
# ----------------------------------------------
prompt = "Fenerbahçe"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Metin üret
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.95
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# ----------------------------------------------
# 1. Model ve Tokenizer Yükleme (Yerel Yol)
# ----------------------------------------------
# DÜZELTME: Model yolunu doğru klasöre ayarlayın
model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114"  # Model ve tokenizer'ın kaydedildiği klasör

# Tokenizer'ı yükle
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Modeli yükle
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.float16
)

# ----------------------------------------------
# 2. Modeli Kullanma
# ----------------------------------------------
prompt = "Fenerbahçe"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Metin üret
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7,
    top_p=0.95
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
import torch
import json
from torch import nn
from transformers import AutoTokenizer  # Sadece tokenizer için (alternatif oluşturabilirsiniz)

# ----------------------------------------------
# 1. Model Yapılandırmasını Yükle (config.json)
# ----------------------------------------------
model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114"

# Config dosyasını oku
with open(f"{model_path}/config.json", "r") as f:
    config = json.load(f)

# ----------------------------------------------
# 2. Model Mimarisini Tanımla (Örnek: GPT2 benzeri)
# ----------------------------------------------
class CustomLM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed_dim = config["hidden_size"]
        self.num_heads = config["num_attention_heads"]
        # ... Diğer katmanları config'ten okuyarak oluştur

    def forward(self, input_ids):
        # ... Modelin forward mantığını uygula
        return logits

# ----------------------------------------------
# 3. Model Ağırlıklarını Yükle (pytorch_model.bin)
# ----------------------------------------------
model = CustomLM(config)
model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin"))
model.eval()

# ----------------------------------------------
# 4. Tokenizer'ı Manuel Oluştur (Örnek)
# ----------------------------------------------
class SimpleTokenizer:
    def __init__(self, vocab_path):
        with open(vocab_path, "r") as f:
            self.vocab = json.load(f)
        self.inverse_vocab = {v: k for k, v in self.vocab.items()}

    def encode(self, text):
        return [self.vocab.get(c, 0) for c in text.split()]

    def decode(self, tokens):
        return " ".join([self.inverse_vocab.get(t, "[UNK]") for t in tokens])

# Tokenizer'ı yükle
tokenizer = SimpleTokenizer(f"{model_path}/vocab.json")

# ----------------------------------------------
# 5. Metin Üretme
# ----------------------------------------------
input_text = "Fenerbahçe will be without the services of"
input_ids = tokenizer.encode(input_text)

with torch.no_grad():
    outputs = model(torch.tensor([input_ids]))
    predicted_token = torch.argmax(outputs[0, -1]).item()

print(tokenizer.decode(input_ids + [predicted_token]))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114/config.json'

In [ ]:
!ls /content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114

adapter_config.json	   README.md	  special_tokens_map.json  tokenizer.model
adapter_model.safetensors  rng_state.pth  tokenizer_config.json    trainer_state.json
optimizer.pt		   scheduler.pt   tokenizer.json	   training_args.bin


In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# ----------------------------------------------
# 1. Ana Model ve Adapter Yolları
# ----------------------------------------------
base_model_name = "google/gemma-2b"  # Orijinal model adı
peft_model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114"

# ----------------------------------------------
# 2. Ana Modeli ve Tokenizer'ı Yükle
# ----------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(base_model_name,token="senin token")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# ----------------------------------------------
# 3. PEFT Adapter'ını Yükle
# ----------------------------------------------
model = PeftModel.from_pretrained(
    base_model,
    peft_model_path,
    adapter_name="default",
    device_map="auto"
)

# ----------------------------------------------
# 4. Modeli Birleştir ve Kullan
# ----------------------------------------------
model = model.merge_and_unload()  # Adapter'ı ana modele entegre et

# Metin üretme örneği
prompt = "Fenerbahçe will be without the services of"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.7
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4

KeyboardInterrupt: 

In [ ]:
import torch
import json
from torch import nn
from transformers import AutoTokenizer  # Sadece tokenizer için (alternatif oluşturabilirsiniz)

# ----------------------------------------------
# 1. Model Yapılandırmasını Yükle (config.json)
# ----------------------------------------------
model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114"

# DÜZELTME: Model dosya yolunu tokenizer'ın kaydedildiği klasöre değiştirin
# tokenizer AutoTokenizer.from_pretrained ile yüklendiğinden,
# config.json dosyasının da aynı dizinde olması gerekir.
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Config dosyasını oku
with open(f"{model_path}/config.json", "r") as f:
    config = json.load(f)

# ----------------------------------------------
# 2. Model Mimarisini Tanımla (Örnek: GPT2 benzeri)
# ----------------------------------------------
class CustomLM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embed_dim = config["hidden_size"]
        self.num_heads = config["num_attention_heads"]
        # ... Diğer katmanları config'ten okuyarak oluştur

    def forward(self, input_ids):
        # ... Modelin forward mantığını uygula
        return logits # DÜZELTME: logits değişkenini tanımlamanız gerekebilir.

# ----------------------------------------------
# 3. Model Ağırlıklarını Yükle (pytorch_model.bin)
# ----------------------------------------------
model = CustomLM(config)
model.load_state_dict(torch.load(f"{model_path}/pytorch_model.bin"))
model.eval()


# ----------------------------------------------
# 5. Metin Üretme
# ----------------------------------------------
input_text = "Fenerbahçe will be without the services of"
input_ids = tokenizer.encode(input_text)

with torch.no_grad():
    outputs = model(torch.tensor([input_ids])) # DÜZELTME: input_ids'i tensöre dönüştürün.
    #DÜZELTME: model.forward fonksiyonunun çıktısına göre logits'i alın
    predicted_token = torch.argmax(outputs[0, -1]).item() #outputs[0, -1] yerine doğru çıktıyı kullanın.

print(tokenizer.decode(input_ids + [predicted_token]))

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114/config.json'

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# ----------------------------------------------
# 1. Ana Model ve Adapter Yolları
# ----------------------------------------------
base_model_name = "google/gemma-2b"  # Orijinal model adı
peft_model_path = "/content/drive/MyDrive/Veri-Kazima/colab/output/checkpoint-7114"

# ----------------------------------------------
# 2. Ana Modeli ve Tokenizer'ı Yükle
# ----------------------------------------------
tokenizer = AutoTokenizer.from_pretrained(base_model_name,token="senin token")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

# ----------------------------------------------
# 3. PEFT Adapter'ını Yükle
# ----------------------------------------------
model = PeftModel.from_pretrained(
    base_model,
    peft_model_path,
    adapter_name="default",
    device_map="auto"
)

# ----------------------------------------------
# 4. Modeli Birleştir ve Kullan
# ----------------------------------------------
model = model.merge_and_unload()  # Adapter'ı ana modele entegre et
# Eğitim sonunda kaydetme
model.save_pretrained("/content/drive/MyDrive/Veri-Kazima/colab/deniz2")
tokenizer.save_pretrained("/content/drive/MyDrive/Veri-Kazima/colab/deniz2")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ERROR:bitsandbytes.cextension:Could not load bitsandbytes native library: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.32' not found (required by /usr/local/lib/python3.11/dist-packages/bitsandbytes/libbitsandbytes_cpu.so)
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 85, in <module>
    lib = get_native_library()
          ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/bitsandbytes/cextension.py", line 72, in get_native_library
    dll = ct.cdll.LoadLibrary(str(binary_path))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 454, in LoadLibrary
    return self._dlltype(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/ctypes/__init__.py", line 376, in __init__
    self._handle = _dlopen(self._name, mode)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^
OSError: /lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4

('/content/drive/MyDrive/Veri-Kazima/colab/deniz2/tokenizer_config.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz2/special_tokens_map.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz2/tokenizer.model',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz2/added_tokens.json',
 '/content/drive/MyDrive/Veri-Kazima/colab/deniz2/tokenizer.json')

In [ ]:
from huggingface_hub import login
login(token="senin token")

In [ ]:
from huggingface_hub import HfApi

# Repo adı (örnek: kullanıcı_adı/model_adı)
repo_name = "denizzhansahin/gemma2bDeneme"

# Modeli yükle
api = HfApi()
api.create_repo(repo_id=repo_name, repo_type="model", private=True)  # Özel repo
api.upload_folder(
    folder_path="/content/drive/MyDrive/Veri-Kazima/colab/deniz2",
    repo_id=repo_name,
    repo_type="model"
)

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/denizzhansahin/gemma2bDeneme/commit/024ba92819267e58f14274e6eb1da80c97403725', commit_message='Upload folder using huggingface_hub', commit_description='', oid='024ba92819267e58f14274e6eb1da80c97403725', pr_url=None, repo_url=RepoUrl('https://huggingface.co/denizzhansahin/gemma2bDeneme', endpoint='https://huggingface.co', repo_type='model', repo_id='denizzhansahin/gemma2bDeneme'), pr_revision=None, pr_num=None)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("denizzhansahin/gemma2bDeneme")
model = AutoModelForCausalLM.from_pretrained("denizzhansahin/gemma2bDeneme")

input_text = "Fenerbahçe"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids)
print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/34.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]